# Fake News Detection Project

### All libraries import

In [1]:
import pandas as pd #for importing dataset
import numpy as nm #for doing mathematical operations 
from sklearn.model_selection import train_test_split as ttp #to break dataset for training and testing
from sklearn.metrics import classification_report # to generate classification report
import re #for data cleaning purpose
import string #to deal with text data
import matplotlib.pyplot as plt #for visualizing dataset

ModuleNotFoundError: No module named 'matplotlib'

### Read Two Data Files

In [2]:
data_true=pd.read_csv("C:\Users\admin\Desktop\Fake news detection\True.csv")
data_fake=pd.read_csv("C:\Users\admin\Desktop\Fake news detection\Fake.csv")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (274628442.py, line 1)

### Display both dataset

In [5]:
data_true.head(5)

NameError: name 'data_true' is not defined

In [ ]:
data_fake.head(5)

### Add Column Name In Both Dataset

In [2]:
data_true["class"] = 1
data_fake["class"] = 0

NameError: name 'data_true' is not defined

In [ ]:
data_true.head(5)

### Taking last 10 values for manual testing from both Datasets

In [ ]:
data_true_manual_testing = data_true.tail(10)
for i in range(21416,21406,-1):
    data_true.drop([i],axis=0, inplace=True)#removing those 10 values from original dataset
    
data_fake_manual_testing = data_fake.tail(10)
for i in range(21416,21406,-1):
    data_fake.drop([i],axis=0, inplace=True)#removing those 10 values from original dataset

### merging these two datasets in single dataframe

In [ ]:
data_manual_testing = pd.concat([data_fake_manual_testing, data_true_manual_testing], axis=0)
data_manual_testing.to_csv("manual_testing.csv ")

### Merging main two datasets

In [ ]:
data_merge = pd.concat([data_fake,data_true],axis=0)
data_merge.head(10)#displaying first 10 values after above operation

### Performing some basic data visualizations

In [ ]:
print(data_merge.groupby(['subject'])['text'].count())
data_merge.groupby(['subject'])['text'].count().plot(kind="bar")
plt.title("Articles per subject",size=20)
plt.xlabel("Category",size=20)
plt.ylabel("Article count",size=20)
plt.show()

In [ ]:
print(data_merge.groupby(['class'])['text'].count())
print("0 = Fake news\n1 = True news")
data_merge.groupby(['class'])['text'].count().plot(kind="pie")
plt.title("Fake news and True News",size=20)
plt.show()

### Removing 3 columns from dataset

In [ ]:
data = data_merge.drop(["title","subject","date"], axis=1)
data.head(10) #displaying remaining dataset 

### Shuffling the dataset for better prediction

In [ ]:
data=data.sample(frac=1)
data.head(10)

### Checking  null values in dataset

In [ ]:
data.isnull().sum()

### Creating data filtering function to remove unwanted data from text

In [ ]:
def filtering(data):
    text=data.lower()
    text=re.sub(r'\[.*?\]','',text)
    text=re.sub(r"\\W"," ",text)
    text=re.sub(r'https?://\s+|www\.S+','',text)
    text=re.sub(r'<.*?>+','',text)
    text=re.sub(r'[%s]'% re.escape(string.punctuation),'',text)
    text=re.sub(r'\w*\d\w*','',text)
    return text

### Filtering data of our text column using above function

In [ ]:
data["text"]= data["text"].apply(filtering)
data.head(10)

### Creating Dependant and independant variables

In [ ]:
x=data["text"]
y=data["class"]

### Spliting dataset for training and testing purpose

In [ ]:
x_train,x_test,y_train,y_test=ttp(x,y,test_size=0.25,random_state=0)

## Vectorizing the text
### The sklearn. feature_extraction module is used to extract features in a format supported by machine learning algorithms from datasets consisting of formats such as text and image

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vector = TfidfVectorizer() # creating object 
#Learn vocabulary and IDF(inverse document frequency), return document-term matrix. 
#IDF returns numerical statics that how the word is important to the document
xv_train=vector.fit_transform(x_train)
xv_test=vector.transform(x_test)

## Classifaction using various classifers
## Logistic Regression
### Calculating probability of event occurence

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LR=LogisticRegression()
LR.fit(xv_train,y_train) #training or fitting the model to the training set

In [ ]:
round(LR.score(xv_test, y_test), 2)

In [ ]:
pred_LR= LR.predict(xv_test) # Predict using the linear model

In [ ]:
print(classification_report(y_test,pred_LR)) #creating classification report

## Creating confusion matrix
### determine the performance of the classification models for a given set of test data. 

In [ ]:
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test,pred_LR)  
print(cm)
import seaborn as sns
sns.heatmap(cm,cmap="BuPu",annot=True)

## Decision Tree Classification
### Non-parametric supervised learning method used for both classification and regression tasks.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
DT = DecisionTreeClassifier()
DT.fit(xv_train,y_train)

In [ ]:
round(DT.score(xv_test, y_test), 2)

In [ ]:
pred_DT = DT.predict(xv_test)

In [ ]:
print(classification_report(y_test,pred_DT))

In [ ]:
cm= confusion_matrix(y_test,pred_DT)  
print(cm)
import seaborn as sns
sns.heatmap(cm,cmap="PiYG",annot=True)

## Random Forest Classifier
### Supervised Machine learning algorithm used for classification, regression, and other tasks using decision trees

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train,y_train)

In [ ]:
RFC.score(xv_test,y_test)

In [ ]:
pred_RFC = RFC.predict(xv_test)

In [ ]:
print(classification_report(y_test,pred_RFC))

In [ ]:
cm= confusion_matrix(y_test,pred_RFC)  
print(cm)
import seaborn as sns
sns.heatmap(cm,cmap="Blues",annot=True)

## Manual Testing

In [ ]:
def output_lable(n):
    if n == 0:
        return "FAKE News"
    elif n == 1:
        return "TRUE News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(filtering) 
    new_x_test = new_def_test["text"]
    new_xv_test = vector.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]),
                                                                                                              output_lable(pred_DT[0]), 
                                                                                                               
                                                                                                              output_lable(pred_RFC[0]))) 
     

## Output

In [81]:
news = str(input())
manual_testing(news)

In [ ]:
news = str(input())
manual_testing(news)